# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [119]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt','wordnet', 'averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Cleandata.db')
df = pd.read_sql("SELECT * FROM Cleandata", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.message
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y = df.iloc[:,4:40]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
for i in range(36):
    print(Y.iloc[:,i].value_counts())

1    20064
0     6116
Name: related, dtype: int64
0    21716
1     4464
Name: request, dtype: int64
0    26062
1      118
Name: offer, dtype: int64
0    15339
1    10841
Name: aid_related, dtype: int64
0    24099
1     2081
Name: medical_help, dtype: int64
0    24869
1     1311
Name: medical_products, dtype: int64
0    25456
1      724
Name: search_and_rescue, dtype: int64
0    25709
1      471
Name: security, dtype: int64
0    25321
1      859
Name: military, dtype: int64
0    26180
Name: child_alone, dtype: int64
0    24511
1     1669
Name: water, dtype: int64
0    23263
1     2917
Name: food, dtype: int64
0    23872
1     2308
Name: shelter, dtype: int64
0    25776
1      404
Name: clothing, dtype: int64
0    25577
1      603
Name: money, dtype: int64
0    25882
1      298
Name: missing_people, dtype: int64
0    25306
1      874
Name: refugees, dtype: int64
0    24988
1     1192
Name: death, dtype: int64
0    22739
1     3441
Name: other_aid, dtype: int64
0    24475
1     1705
Name:

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Punctuation Removal
    
    tokens=word_tokenize(text)   # tokenization
    
    lemmatizer=WordNetLemmatizer()  
    
    clean_tokens =[]
    
    for tok in tokens:
        
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)
        
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',  CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [9]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred=pipeline.predict(X_test)

In [13]:
colname=list(Y_test.columns)
print(colname)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [80]:
# modularize the classification_report
def classreport(Y_test, Y_pred):
    """
    modularize the classification_report
    
    """
    colname=list(Y_test.columns)
    
    Y_pred2=pd.DataFrame(Y_pred, index=Y_pred[:,0])
    
    target_names=['0','1']

    a=classification_report(Y_test, Y_pred2, target_names=target_names)
    
    avg_total_f1=float(a[-17:-9])
    
    return avg_total_f1

In [26]:
print(classification_report(Y_test, Y_pred2, target_names=target_names))

             precision    recall  f1-score   support

          0       0.83      0.93      0.88      4997
          1       0.84      0.38      0.53      1127

avg / total       0.74      0.46      0.51     20842



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
classreport(Y_test, Y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 2
  .format(len(labels), len(target_names))


0.928611111111111

In [81]:
# define a evalution function
def eva_fun(Y_test, Y_pred):
    '''
    combine the total accuracy and f1_score in the classification report
    '''
    print('avg_f1_score', classreport(Y_test, Y_pred))
    
    return result

In [82]:
eva1=eva_fun(Y_test, Y_pred)

total_accuracy 0.942513368984
avg_f1_score 0.51


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 36, does not match size of target_names, 2
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## cross validation and f1 score

In [94]:
# Cross-validate for pipeline 1
cross_val_score(pipeline, X, Y, cv=3, scoring='f1')

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [96]:
cv_results = cross_validate(pipeline, X, Y, cv=3)

In [97]:
cv_results['test_score']

array([ 0.25633093,  0.17990146,  0.17591107])

In [116]:
cv_results

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.81975722,  56.14768934,  55.53346372]),
 'score_time': array([ 5.26799726,  6.08702493,  6.6382978 ]),
 'test_score': array([ 0.25633093,  0.17990146,  0.17591107]),
 'train_score': array([ 0.79751332,  0.8295422 ,  0.82978114])}

In [98]:
cv_results2 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1'))

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [108]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [107]:
Y.related.value_counts()

1    20064
0     6116
Name: related, dtype: int64

In [109]:
for i in Y.columns:
    print(Y[i].value_counts())

1    20064
0     6116
Name: related, dtype: int64
0    21716
1     4464
Name: request, dtype: int64
0    26062
1      118
Name: offer, dtype: int64
0    15339
1    10841
Name: aid_related, dtype: int64
0    24099
1     2081
Name: medical_help, dtype: int64
0    24869
1     1311
Name: medical_products, dtype: int64
0    25456
1      724
Name: search_and_rescue, dtype: int64
0    25709
1      471
Name: security, dtype: int64
0    25321
1      859
Name: military, dtype: int64
0    26180
Name: child_alone, dtype: int64
0    24511
1     1669
Name: water, dtype: int64
0    23263
1     2917
Name: food, dtype: int64
0    23872
1     2308
Name: shelter, dtype: int64
0    25776
1      404
Name: clothing, dtype: int64
0    25577
1      603
Name: money, dtype: int64
0    25882
1      298
Name: missing_people, dtype: int64
0    25306
1      874
Name: refugees, dtype: int64
0    24988
1     1192
Name: death, dtype: int64
0    22739
1     3441
Name: other_aid, dtype: int64
0    24475
1     1705
Name:

In [110]:
cv_results3 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1_samples'))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [111]:
cv_results3

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.5671289 ,  55.84412789,  56.3019073 ]),
 'score_time': array([ 5.35025096,  6.01594067,  6.80977654]),
 'test_score': array([ 0.34850601,  0.50005417,  0.48898858]),
 'train_score': array([ 0.78385294,  0.70733705,  0.73485949])}

In [114]:
cv_results4 = cross_validate(pipeline, X, Y, cv=3)

In [115]:
cv_results4

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 65.675421  ,  54.80811524,  55.64870548]),
 'score_time': array([ 5.22247553,  6.03239512,  6.69437766]),
 'test_score': array([ 0.26515412,  0.21782972,  0.16743067]),
 'train_score': array([ 0.79911763,  0.8278233 ,  0.82703105])}

In [117]:
cv_results5 = cross_validate(pipeline, X, Y, cv=3, scoring=('f1_weighted'))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

In [118]:
cv_results5

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([ 64.93734765,  56.23888016,  56.19602537]),
 'score_time': array([ 5.31334949,  6.07269669,  6.73505902]),
 'test_score': array([ 0.37522138,  0.4534976 ,  0.40630495]),
 'train_score': array([ 0.95405755,  0.95792917,  0.96177909])}

In [120]:
f1_score(Y_test, Y_pred2, average='macro') 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.21513822820998288

In [121]:
f1_score(Y_test, Y_pred2, average='micro') 

0.58444546709618039

In [122]:
f1_score(Y_test, Y_pred2) 

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [123]:
f1_score(Y_test, Y_pred2, average='weighted') 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.51304767444845789

### Implement the TextLengthExtractor class

In [127]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return len(tokenize(X))

In [125]:
a='i am superman haha'

In [128]:
length=TextLengthExtractor()

In [129]:
length.transform(a)

4

### Implement the StartingVerbExtractor class

In [130]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

In [131]:
b='love you haha haha'

In [132]:
verb=StartingVerbExtractor()

In [133]:
verb.transform(b)

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/lib/nltk_data'
**********************************************************************


### Feature Union

In [134]:
pipeline5 = Pipeline([
    ('features', FeatureUnion([
    
        ('nlp_pipeline', Pipeline([
            ('vect',  CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('text_length', TextLengthExtractor())
        
    ])),
        
    
    ('mclf5', MultiOutputClassifier(RandomForestClassifier()))
])

In [135]:
pipeline5.fit(X_train, Y_train)

TypeError: expected string or bytes-like object

In [ ]:
pipeline6 = Pipeline([
    ('features', FeatureUnion([
    
        ('nlp_pipeline', Pipeline([
            ('vect',  CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor()),
        
        ('text_length', TextLengthExtractor())
        
    ])),
        
    
    ('mclf6', MultiOutputClassifier(RandomForestClassifier()))
])